In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
import json
path = '/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/CAPP30239_FA22/Final_Project'
# path = '/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/EatingHabitsOfNewYorkers/Eating-Habits-of-New-Yorkers'

Intro: Analysis fo restaurants in New York because ....

Visual 1: Bubble chart of all categories with price (mode) shown as color 

Visual 2: Diving deeper into top ten categories and checking their prices --> ring charts of categories and prices 

Visual 3: Wanted to see whether ratings are affected by price --> heatmap of ratings, categories and prices

Visual 4: (optional?) Interesting analysis. Overall, from a hgiher level, these are the average ratings by category --> horizontal bar chart. 
    Possibly make this one interactive. Ratings by category and ratings by price. 
    
Visual 5: Where are these restaurants located? Categories --> drop down menu with categories. Markers show up on map with details about restaurnats. 

In [3]:
# ### CODE TO CLEAN SCRAPED CENSUS DATA ###

# income = pd.read_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/DataViz_projectdata/income_data.csv')

# cols_to_drop = ['New York!!Households!!Estimate',
#        'New York!!Households!!Margin of Error', 'New York!!Families!!Estimate',
#        'New York!!Families!!Margin of Error',
#        'New York!!Married-couple families!!Estimate',
#        'New York!!Married-couple families!!Margin of Error',
#        'New York!!Nonfamily households!!Estimate',
#        'New York!!Nonfamily households!!Margin of Error']

# income.drop(columns = cols_to_drop, inplace = True)


# cols_to_drop = []
# for colname in income.columns[1:]:
#     name_split = colname.split('!!')
#     print(name_split)
#     if name_split[1] != 'Households':
#         cols_to_drop.append(colname)
#     if name_split[2] != 'Estimate':
#         cols_to_drop.append(colname)

# income.drop(columns = cols_to_drop, inplace = True)
# income


# zipcodes_lst = []

# for colname in income.columns[1:]:
#     name_split = colname.split('!!')
#     zipcode = name_split[0].split(' ')[1]
#     zipcodes_lst.append(zipcode)

# zipcodes_lst = ['Label (Grouping)'] + zipcodes_lst

# income.columns = zipcodes_lst
# income.set_index('Label (Grouping)', inplace = True)


# income_transposed = income.T
# income_df = income_transposed.loc[:, ['Total', 'Median income (dollars)', 'Mean income (dollars)']]
# income_df.reset_index(inplace = True)
# income_df.rename(columns = {'index': 'Zipcode'}, inplace = True)

# # income_df.to_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/DataViz_projectdata/income_by_zipcode.csv', index = False)


In [4]:
# READ ALL CLEAN FILES 

# yelp = pd.read_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/DataViz_projectdata/yelp_rest_data.csv')
yelp = pd.read_csv(os.path.join(path, 'data', 'yelp_rest_data.csv'))
# income = pd.read_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/DataViz_projectdata/income_by_zipcode.csv')
income = pd.read_csv(os.path.join(path, 'data', 'income_by_zipcode.csv'))


In [5]:
yelp['Zipcode'] = yelp['Zipcode'].fillna(0)
yelp['Zipcode'] = yelp['Zipcode'].astype(int)
# yelp

In [6]:
print(f'Number of zipcodes in yelp dataset: {len(yelp["Zipcode"].unique())}')
print(f'Number of zipcodes in income dataset: {len(income["Zipcode"].unique())}')


Number of zipcodes in yelp dataset: 51
Number of zipcodes in income dataset: 47


In [7]:
data = pd.merge(yelp, income, how = 'left', left_on = 'Zipcode', right_on = 'Zipcode')
# data.to_csv(os.path.join(path, 'yelp_and_income.csv'), index = False)
merged_df = pd.read_csv(os.path.join(path, 'yelp_and_income.csv'))
merged_df = merged_df[merged_df['Zipcode'] != 0]

In [9]:
# merged_df.head(20)

In [10]:
### CREATING DATASET FOR PACKED BUBBLE CHART 
agg_dict = {'Name': 'size', 'Price': pd.Series.mode}
price_lst = ['$', '$$', '$$$', '$$$$']
# bubble_df = merged_df.groupby('Category')['Name'].size().reset_index()
bubble_df = merged_df.groupby('Category').agg(agg_dict).reset_index()
bubble_df = bubble_df[bubble_df['Price'].isin(price_lst)]
bubble_df.rename(columns = {'Name': 'Count'}, inplace = True)

# bubble_df.to_csv(os.path.join(path, 'bubble.csv'), index = False)
bubble_df = pd.read_csv(os.path.join(path, 'bubble.csv'))
bubble_df

,Category,Count,Price
0,African,2,$$
1,American (New),60,$$
2,American (Traditional),8,$$$
3,Asian Fusion,5,$$$
4,Australian,5,$$
...,...,...,...
89,Turkish,1,$$$
90,Vegetarian,2,$$
91,Venues & Event Spaces,1,$$
92,Vietnamese,16,$$


In [20]:
barchart2 = bubble_df[['Category', 'Count']].sort_values('Count', ascending = False).reset_index(drop=True)
barchart2 = barchart2.iloc[:11]
barchart2

,Category,Count
0,Italian,61
1,American (New),60
2,Japanese,30
3,Korean,27
4,French,22
5,Mexican,22
6,Cocktail Bars,22
7,Thai,21
8,Chinese,21
9,Bars,19


In [14]:
### SELECT TOP TEN CATEGORIES BY COUNT 

cat_counts = data['Category'].value_counts(ascending = False)
top_10_lst = list(cat_counts.head(12).index)
top_10_cats_df = data[data['Category'].isin(top_10_lst)]
top_10_cats_df 

,Name,Category,Latitude,Longitude,Price,Rating,Zipcode,Total,Median income (dollars),Mean income (dollars)
0,Da Andrea,Italian,40.736218,-73.995970,$$,4.5,10011,"28,775","136,208","210,553"
1,LoveMama,Thai,40.730409,-73.986127,$$,4.5,10003,"25,158","129,981","211,756"
5,The Cabin NYC,American (New),40.723930,-73.983830,$$,4.0,10009,"29,356","68,220","108,017"
7,Antidote,Cocktail Bars,40.714253,-73.965448,$$,5.0,11249,NaN,NaN,NaN
8,Thursday Kitchen,Korean,40.727610,-73.983730,$$,4.5,10009,"29,356","68,220","108,017"
...,...,...,...,...,...,...,...,...,...,...
940,After Life,Cocktail Bars,40.703030,-73.933380,NaN,5.0,0,NaN,NaN,NaN
941,Velvet Brooklyn,Cocktail Bars,40.710085,-73.962579,$$,4.5,11211,"40,821","90,871","128,607"
942,A-un Brooklyn,Japanese,40.704160,-73.927990,NaN,5.0,0,NaN,NaN,NaN
944,Clinton Hall - FiDi,Bars,40.708030,-74.014869,$$,3.5,10006,"1,811","185,268","226,459"


#### Chart idea: Average Ratings by Zipcode

In [15]:
len(data['Zipcode'].unique())
data['Median income (dollars)'] = data['Median income (dollars)'].str.replace(',', '')
data['Median income (dollars)'] = data['Median income (dollars)'].str.replace('+', '')
data['Median income (dollars)'] = data['Median income (dollars)'].astype(float)

/var/folders/7x/ln_3wt0j7v9dkyhj_b2t6v480000gn/T/ipykernel_52117/2809889007.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Median income (dollars)'] = data['Median income (dollars)'].str.replace('+', '')


#### Chart Idea: Ring for each category with each donut showing percentage of expensive and least expensive restaurants 

In [13]:
top_10_cats_df

cat_price_df = top_10_cats_df.groupby(['Category', 'Price'])['Name'].size().reset_index()

def get_pcnt(df):

    df['Percentage'] = (df['Name']/df['Name'].sum()) * 100
    df['Percentage'] = np.round(df['Percentage'], 1)
    return df

dfs_lst = []

for _, group in cat_price_df.groupby(['Category']):
    df = get_pcnt(group)
    dfs_lst.append(df)


cat_price_df = pd.concat(dfs_lst)
cat_price_df.to_csv(os.path.join(path, 'Cat_price_pcnt.csv'), index = False)
cat_price_df = pd.read_csv(os.path.join(path, 'Cat_price_pcnt.csv'))
cat_price_df.rename(columns = {'Name':'Count'}, inplace = True)


cat_price_json = [{'Category': 'American (New)', 'Values': [{'category': '$$', 'amount': 56.666667}, {'category': '$$$', 'amount': 31.666667}, 
                {'category': '$$$$', 'amount': 11.666667}]},
        {'Category' :'Chinese', 'values': [{'category': '$', 'amount': 38.095238}, {'category': '$$', 'amount': 52.380952}, 
            {'category': '$$', 'amount': 4.761905}, {'category': '$$', 'amount': 4.761905}]},
        {'Category': 'Cocktail Bars', 'values' :[{'category': '$$', 'amount': 86.3636368}, {'category': '$$$', 'amount': 13.636364}]},
        {'Category': 'French', 'values': [{'category': '$$', 'amount': 50.000000}, {'category': '$$$', 'amount': 45.454545}, 
            {'category': '$$$$', 'amount': 4.545455}]},
        {'Category': 'Italian', 'amount': [{'category': '$', 'amount': 1.639344}, {'category': '$$', 'amount': 57.377049}, 
            {'category': '$$$', 'amount': 37.704918}, {'category': '$$$$', 'amount': 3.278689}]},
        {'Category': 'Japanese', 'amount': [{'category': '$', 'amount': 3.333333}, {'category': '$$', 'amount': 70.000000}, 
            {'category': '$$$', 'amount': 16.666667}, {'category': '$$$$', 'amount': 10.000000}]},
        {'Category': 'Korean', 'amount': [{'category': '$', 'amount': 3.703704}, {'category': '$$', 'amount': 85.185185}, 
            {'category': '$$$', 'amount': 7.407407}, {'category': '$$$$', 'amount': 3.703704}]},
        {'Category': 'Mexican', 'amount': [{'category': '$$', 'amount': 81.818182}, {'category': '$$$', 'amount': 18.181818}]},
        {'Category': 'Seafood', 'amount': [{'category': '$$', 'amount': 66.666667}, {'category': '$$', 'amount': 33.333333}]},
        {'Category': 'Thai', 'amount': [{'category': '$$', 'amount': 90.4761907}, {'category': '$$', 'amount': 9.523810}]}
]

/var/folders/7x/ln_3wt0j7v9dkyhj_b2t6v480000gn/T/ipykernel_40721/3970144151.py:13: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in cat_price_df.groupby(['Category']):


In [16]:
barchart = top_10_cats_df.groupby('Category')['Name'].size().sort_values(ascending = False).reset_index()
barchart.rename(columns = {'Name':'Count'}, inplace = True)
barchart

# barchart.to_csv(os.path.join(path, 'barchart.csv'), index = False)
# barchart = pd.read_csv(os.path.join(path, 'barchart.csv'))
# barchart

,Category,Count
0,American (New),82
1,Italian,80
2,Japanese,44
3,Korean,39
4,French,35
5,Mexican,34
6,Thai,32
7,Cocktail Bars,30
8,Chinese,29
9,Seafood,27


In [15]:
# cat_price_df.to_csv(os.path.join(path, 'cat_price_pcnt.csv'), index = False)
# cat_price_df = pd.read_csv(os.path.join(path, 'cat_price_pcnt.csv'))
# cat_price_df

# for cat in list(cat_price_df['Category'].unique()):
#     df = cat_price_df[cat_price_df['Category'] == cat]
#     df.to_csv(os.path.join(path, f'{cat}_price_pcnt.csv'), index = False)

#### Chart Idea: Bar chart of price and ratings 

In [16]:
#BAR CHART OF PRICE AND RATINGS 

price_ratings = data.groupby('Price')['Rating'].mean().reset_index()
price_ratings


,Price,Rating
0,$,4.387500
1,$$,4.198312
2,$$$,4.174242
3,$$$$,4.375000


In [17]:
category_ratings = data.groupby('Category')['Rating'].mean().reset_index().sort_values('Rating', ascending = False).reset_index(drop=True)
category_ratings.head()

,Category,Rating
0,Ethiopian,5.000000
1,Chicken Wings,5.000000
2,Sports Bars,5.000000
3,New Mexican Cuisine,4.750000
4,African,4.666667


In [18]:
len(data[data['Category'] == 'Thai'])

32

In [19]:
# Start writeup about data cleaning - drop restaurants with less then 50 ratings 
# Build map today 

cat_counts = data['Category'].value_counts(ascending = False)
top_10_lst = list(cat_counts.head(10).index)
top_10_cats_df = data[data['Category'].isin(top_10_lst)]
top_10_cats_df 

# array_for_map_df = top_10_cats_df[['Longitude', 'Latitude', 'Category']] ### old code 
array_for_map_df = top_10_cats_df[['Longitude', 'Latitude', 'Category', 'Name', 'Rating', 'Price']]

cat_color_dict = {'Italian':'red', 'Thai':'blue', 'American (New)':'purple', 'Cocktail Bars':'pink', 'Korean':'green', 'Mexican':'yellow', 'Japanese':'black', 'French':'orange', 'Chinese': 'brown', 'Seafood':'darkturquoise'}

array_for_map_df['Color'] = array_for_map_df['Category'].map(cat_color_dict)
# array_for_map_df.drop(columns = ['Category'], inplace = True)
all_cats_array_for_map = array_for_map_df.to_json(orient= 'records')
# all_cats_array_for_map

# CODE FOR JUST ONE CATEGORY 
# cat_map_df = array_for_map_df[array_for_map_df['Category'] == 'French']

# array_for_map = cat_map_df.to_json(orient='records')
# array_for_map

#### CODE FOR MAP_DATA.JSON USED IN MAP.JS ####
# with open("map_data.json", "w") as outfile:
#     json.dump(all_cats_array_for_map, outfile)
# json_lst = json.loads(all_cats_array_for_map)
# json_lst

# all_cats_array_for_map

/var/folders/7x/ln_3wt0j7v9dkyhj_b2t6v480000gn/T/ipykernel_40721/1054647289.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  array_for_map_df['Color'] = array_for_map_df['Category'].map(cat_color_dict)


In [22]:
### CONVERT TO JSON 

for cat in list(array_for_map_df['Category'].unique()):
    cat_df = array_for_map_df[array_for_map_df['Category'] == cat]
    array_for_map = array_for_map_df.to_json(orient='records')
    with open(f"{cat}_map_data.json", "w") as outfile:
        json.dump(array_for_map, outfile)


In [24]:
# agg_dict = {'Rating': np.mean, 'Name': np.size, 'Price': pd.Series.mode}
agg_dict = {'Rating': np.mean, 'Name': np.size}

stacked_bar_df = top_10_cats_df.groupby(['Category', 'Price']).agg(agg_dict).reset_index().rename(columns = {'Name': 'Count'})
# stacked_bar_df.to_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/DataViz_projectdata/stacked_bar.csv', index = False)
stacked_bar_df


,Category,Price,Rating,Count
0,American (New),$$,4.058824,34
1,American (New),$$$,4.078947,19
2,American (New),$$$$,4.357143,7
3,Chinese,$,4.375000,8
4,Chinese,$$,4.227273,11
5,Chinese,$$$,4.000000,1
6,Chinese,$$$$,4.000000,1
7,Cocktail Bars,$$,4.342105,19
8,Cocktail Bars,$$$,4.166667,3
9,French,$$,4.272727,11


In [26]:
stacked_bar_df = pd.read_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/CAPP30239_FA22/Final_Project/stacked_bar_.csv')
# stacked_bar_pivot = stacked_bar_df.pivot(index = 'Category', columns = 'Price', values = 'Rating').reset_index()
# stacked_bar_pivot

stacked_bar_df[stacked_bar_df['Category'] == 'Mexican']

,Category,Price,Rating,Count
24,Mexican,$$,4.194444,18
25,Mexican,$$$,4.250000,4


In [27]:
# stacked_bar_pivot.to_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/CAPP30239_FA22/Final_Project/stacked_bar.csv', index = False)

df = pd.read_csv('/Users/irsaashraf/Desktop/UChicago/Fall_22/DataViz/CAPP30239_FA22/Final_Project/stacked_bar.csv')
df

,Category,$,$$,$$$,$$$$
0,American (New),NaN,4.058824,4.078947,4.357143
1,Chinese,4.375,4.227273,4.000000,4.000000
2,Cocktail Bars,NaN,4.342105,4.166667,NaN
3,French,NaN,4.272727,4.200000,4.000000
4,Italian,4.500,4.200000,4.130435,4.250000
5,Japanese,4.500,4.166667,4.200000,4.833333
6,Korean,4.000,4.347826,4.250000,5.000000
7,Mexican,NaN,4.194444,4.250000,NaN
8,Seafood,NaN,4.083333,4.250000,NaN
9,Thai,NaN,4.342105,4.500000,NaN


In [28]:
### CHART 2: bar plot of category and ratings 

ratings_cat = top_10_cats_df.groupby('Category')['Rating'].mean().reset_index().sort_values('Rating', ascending = False)
ratings_cat

,Category,Rating
9,Thai,4.484375
6,Korean,4.384615
2,Cocktail Bars,4.350000
3,French,4.314286
5,Japanese,4.306818
7,Mexican,4.250000
4,Italian,4.243750
0,American (New),4.225610
1,Chinese,4.224138
8,Seafood,4.222222


In [29]:
ratings_cat.to_csv(os.path.join(path, 'ratings_cat.csv'), index = False)

ratings_cat = pd.read_csv(os.path.join(path, 'ratings_cat.csv'))
ratings_cat

,Category,Rating
0,Thai,4.484375
1,Korean,4.384615
2,Cocktail Bars,4.350000
3,French,4.314286
4,Japanese,4.306818
5,Mexican,4.250000
6,Italian,4.243750
7,American (New),4.225610
8,Chinese,4.224138
9,Seafood,4.222222
